## Load the rasterdata from pickle or USGS into Python

`EOdal SceneCollections` can be made persistent by storing them as serialized pickled objects on disk (and can be loaded from there). Thus, significant amounts of time can be saved as data extraction and processing routines have not necessarily to be repeated.
If the Scenes are extracted for the first time this will take some time depending on the extend and numbers of scenes found per time interval.

In [ ]:
fpath: Path = Path(base_dir.joinpath(f'scenes/all_scenes_{shapefile_name[:-4]}_{str(time_start)[:-9]}_{str(time_end)[:-9]}.pkl'))

if fpath.exists():
    print(f"The SceneCollection is loaded from the pickle file at: {fpath}")
    mapper.data = SceneCollection.from_pickle(stream=fpath)
    print(mapper.data)

else:
    print(f"The file {fpath} does not yet exist thus the SceneCollection will be extracted from the USGS archive.")

    # We tell EOdal how to load the Landsat scenes using `Landsat.from_usgs`and pass on some kwargs, e.g., the selection of bands we want to read. 
    # In addition, we tell EOdal to mask out clouds and shadows and the fly while reading the data using the qa_pixel band (therefore, we set the `read_qa` flag to True.
    scene_kwargs = {
        'scene_constructor': Landsat.from_usgs,
        'scene_constructor_kwargs': {
            'band_selection': ['blue', 'green', 'red', 'nir08', 'swir16', 'swir22'],
            'read_qa': True},
        'scene_modifier': prepocess_landsat_scene,
        'scene_modifier_kwargs': {}
        }

    # now we load the scenes
    mapper.load_scenes(scene_kwargs=scene_kwargs)

    with open(fpath, 'wb+') as dst:
        dst.write(mapper.data.to_pickle())